# Tarefa de Casa 

O link abaixo disponibiliza um dataset de avaliações de filmes, contendo o texto da avaliação e se foi positiva ou negativa:
https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


Fazer um pipeline completo de NLP para classificação com as seguintes características:
1 - Devem haver 3 classificadores, sendo pelo menos 1 de Deep Learning
2 - Justificativa de todos os artíficios utilizados, regex, modelos, camadas.

Se concentrem na documentação e justificativa das escolhas. O importante é mostrar que entendeu o conteúdo através da pesquisa e não obter a melhor métrica.

Caso tenham dúvidas:
leohb2@gmail.com
github.com/Leothi

In [0]:
import cv2

import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import spacy
import spacy.cli

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

nlp = spacy.load("en_core_web_sm")


LOCAL = False

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
from os import path
import zipfile

if not path.exists('dataset'):
    link = 'https://drive.google.com/open?id=1lR9wldgNYUNN0HyPC8mBdltlzN4fox6m'

    fluff, file_id = link.split('=')
    downloaded = drive.CreateFile({'id':file_id})
    downloaded.GetContentFile('dataset.zip')

    archives = zipfile.ZipFile('dataset.zip')

    archives.extractall('dataset')


In [0]:
df = pd.read_csv('dataset/IMDB Dataset.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null object
dtypes: object(2)
memory usage: 781.4+ KB


In [0]:
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


O bloco a seguir é responsável por limpar os textos presentes na coluna 'review'. Essa limpeza é necessária, visto que existem tags de html no meio do texto e para a melhor obtenção de informações sobre as palavras, deve-se retirar palavras que não agregam muita informação (stopwords) bem como retirar qualquer outro conjunto de caracteres que não sejam palavras, como acentos, pontuação etc.

Alem disso, é retirada informações sobre o texto referente ao tamanho médio tanto de palavras quanto de sentenças de cada review do dataframe.

In [0]:
import unicodedata
import re


# Função responsável por limpar o texto e deixá-lo todo em lowercase
def clean_text(text):
    # Lista de stopwords em ingles
    en_stopwords = set(stopwords.words('english'))
    
    # Regex para retirada de 'não palavras'
    re_only_words = re.compile(r'\W')

    text = text.lower()

    # Retirada de stopwords
    text = ' '.join(word for word in text.split() if word not in en_stopwords)
    
    # Retirada de acentos
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")

    # Retiradas de 'não palavras'
    text = re_only_words.sub(' ', text)


    # print(text)
    return text


# Função responsável pela retirdada de tags existentes dentro do texto
def clean_tags(text):
    re_tags = re.compile(r'<(.*?)>')
    
    # Retirada de tags
    text = re_tags.sub(' ', text)
    return text

# Função responsável por obter o tamanho médio das palavras em caracteres
def avg_word(text):
    re_only_words = re.compile(r'\W')


    text = re_only_words.sub(' ', text)
    total = 0

    for word in text.split():
        total += len(word)
    return total/len(text.split())

# Função responsável por obter o tamanho médio das sentenças em caracteres
def avg_sent(text):
    doc = nlp(text)

    total = 0
    for sent in list(doc.sents):
        total += len(sent)
    return(total/len(list(doc.sents)))


# Apenas roda toda a parte de limpeza e obtenção de features caso a variável LOCAL esteja como Verdadeiro
# Caso contrário, um arquivo csv já com esses dados processados será utilizado mais a frente
if (LOCAL):
    print('Getting rid of tags...')
    df.review = df.review.apply(clean_tags)

    print('Getting avg word length...')
    df['avg_word_lenght'] = df.review.apply(avg_word)

    print('Getting avg sentences length...')
    df['avg_sent_lenght'] = df.review.apply(avg_sent)

    print('Cleaning text...')
    df.review = df.review.apply(clean_text)

    df.head(5)

In [0]:
# Adição das colunas referentes às features a serem obtidas do texto
features = ['ADJ', 'ADV', 'DET', 'NOUN', 'PRON', 'PROPN' ,'VERB', 'JJ', 'JJR', 'JJS']
for feature in features:
    df[feature] = 0

In [0]:
import progressbar
from collections import Counter


# Preenchimento das novas colunas no dataframe com os sues respectivos valores
# (Apenas roda caso LOCAL seja True, caso contrário, baixa-se o .csv já processado)
if(LOCAL):
    removables = ['CCONJ', 'SPACE', 'NUM']

    true_bar = progressbar.ProgressBar(maxval=df.shape[0], \
    widgets=[progressbar.Bar('#', '[', ']'), '0/{}   '.format(df.shape[0], progressbar.Percentage)])
    true_bar.start()
    i = 0

    for index, row in df.iterrows():
        i += 1
        true_bar.widgets[1] = ' {}/{}'.format(i,df.shape[0])
        true_bar.update(i)

        doc = nlp(row.review)
        c = Counter(([token.pos_ for token in doc]))
        d = Counter(([token.tag_ for token in doc if token.pos_ == 'ADJ']))
        c.update(d)

        for key in removables:
            del c[key]
        for key in c:
            row[key] = c[key]
        df.iloc[index] = row
    df.to_csv('IMDB Features.csv', index=False)
else:
  features_csv = 'https://drive.google.com/open?id=1qq1nXCQ9o6LMW1c8LjkH7gCtBWQKel1E'

  file_id = features_csv.split('=')[1]
  downloaded = drive.CreateFile({'id':file_id})
  downloaded.GetContentFile('IMDB Features.csv')


df = pd.read_csv('IMDB Features.csv')
df.head()

,review,sentiment,avg_word_lenght,avg_sent_lenght,ADJ,ADV,DET,NOUN,PRON,PROPN,VERB,JJ,JJR,JJS
0,one reviewers mentioned watching 1 oz episode ...,positive,4.347134,24.133333,32,15,1,75,8,0,42,29,3,0
1,wonderful little production filming technique...,positive,4.881250,20.555556,15,13,2,38,0,0,20,15,0,0
2,thought wonderful way spend time hot summer we...,positive,4.244048,32.500000,21,5,0,38,2,1,20,21,0,0
3,basically there s family little boy jake thi...,negative,4.125926,17.444444,7,8,0,33,1,0,18,6,1,0
4,petter mattei s love time money visually stu...,positive,4.456140,21.500000,23,4,1,63,4,0,23,22,0,1


O bloco de código a seguir é responsável por stemizar e lematizar o texto da coluna 'review' já limpado. Isso ajuda algorítmos a extrair informações de texto, visto que retira informações temporais ou de concordância das palavras, permitindo que o texto final seja o mais cru e auditavel possível. 

In [0]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

# Função que retorna o texto stemizado
def get_stemmed_text(text):
    stemmer = PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

# Função que retorna o texto lematizar
def get_lemmatized_text(text):
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text


df.review = df.review.apply(get_stemmed_text)
df.review = df.review.apply(get_lemmatized_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


KeyboardInterrupt: ignored

In [0]:
df['clean review'] = df.review
df.drop(['review'], axis=1, inplace=True)

In [0]:
df.head()

In [0]:
# Função responsável por codificar com um valor numérico o 'sentiment' de cada review do dataset
def code_sentiment(sentiment):
    if (sentiment == 'positive'):
        return 1
    return 0

df.sentiment = df.sentiment.apply(code_sentiment)

In [0]:
# import seaborn as sns

# sns.pairplot(df.drop(['review'],axis=1), hue='sentiment')

In [0]:
df.info()

In [0]:
df.head(5)

A partir desse novo dataframe apenas com o review lematizado e stemizado, é feito a contagem das 1500 palavras que mais ocorrem bem como o TF-IDF das 1500 palavras com maior valor quando levado em conta todos os campos da coluna 'clean review'

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True, max_features=1500)
cv.fit(df['clean review'])
vectorizer = cv.transform(df['clean review'])

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(
                    sublinear_tf = True,
                    max_features = 1500)

train_tv = tv.fit_transform(df['clean review'])

In [0]:
train_tv

In [0]:
# dist = np.sum(train_tv, axis=0)

In [0]:
tfidf_df = pd.DataFrame.sparse.from_spmatrix(train_tv, columns=tv.get_feature_names())

In [0]:
count_df = pd.DataFrame.sparse.from_spmatrix(vectorizer)

In [0]:
df.info()

In [0]:
# Junção dos dois dataframes criados para o CountVectorizer e o TFIDFVectorizer com o dataframe original
# em um novo dataframe
new_df = df.join(count_df)
new_df = new_df.join(tfidf_df)

In [0]:
new_df.info()

In [0]:
new_df.fillna(0, inplace=True)

A partir desse novo dataframe, podemos enfim separá-lo em treino e teste, instanciar um modelo e testá-lo. Isso é feito nos blocos de código a seguir.

In [0]:
from sklearn.model_selection import train_test_split

X_train, x_test, Y_train, y_test = train_test_split(new_df.drop(['sentiment', 'clean review'], axis=1), new_df['sentiment'], test_size=0.3, random_state=42)

### Random Forest Classifier

Escolheu-se iniciar com o modelo de Random Forest. A seguir, apresenta-se a criação do modelo, e o uso dos conjuntos de treino e teste para o seu treino e sua validação.

In [0]:
from sklearn.ensemble import RandomForestClassifier

# Criação do dataframe para o modelo com todos os parâmetros default
model = RandomForestClassifier(random_state=42)

model.fit(X_train, Y_train)

In [0]:
# Obtenção da predição a partir do conjunto de teste com o modelo criado
pred = model.predict(x_test)

In [0]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score, accuracy_score

matrix = confusion_matrix(y_test, pred)

fig, ax = plt.subplots()

sns.heatmap(matrix, annot=True, fmt='d')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
fig.show()

# Verificação da accuracy, recall e precision do modelo a partir da predição com valores de teste
print('Model Accuracy:\t\t', accuracy_score(y_test, pred))
print('Model Recall:\t\t', recall_score(y_test, pred))
print('Model Precision:\t', precision_score(y_test, pred))

Foi escolhido o RandomForestClassifier por ser muito adaptável a diversos tipos de problemas de classificação. os valores obtidos expostos acima se mostram favoráveis, com todos os parâmetros acima de 80%.

### Logistic Regression

A seguir, escolheu-se aplicar o regessor logístico para a classificação. Nos blocos de códigos abaixo apresentam-se a criação do modelo, e o uso dos conjuntos de treino e teste para o seu treino e sua validação.

In [0]:
from sklearn.linear_model import LogisticRegression

# Criação do modelo com todos os parâmetros default
logistic_reg_clf = LogisticRegression(random_state=42)

# Treino do modelo a partir do conjunto de dados separado para treino e suas labels
logistic_reg_clf.fit(X_train, Y_train)

In [0]:
pred = logistic_reg_clf.predict(x_test)

In [0]:
matrix = confusion_matrix(y_test, pred)

fig, ax = plt.subplots()

sns.heatmap(matrix, annot=True, fmt='d')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
fig.show()

# Verificação da accuracy, recall e precision do modelo a partir da predição com valores de teste
print('Model Accuracy:\t\t', accuracy_score(y_test, pred))
print('Model Recall:\t\t', recall_score(y_test, pred))
print('Model Precision:\t', precision_score(y_test, pred))

Os resultados obtidos acima também foram favoráveis, até superiores do que os obtidos com o modelo de Random Forest. Sua aplicação para quantidade de variáveis adicionadas ao modelo parece lidar bem com esse modelo. 

### Deep Learning

A seguir, é apresentado a criação de um modelo de Deep Learning com 7 camadas.

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

# Criação de um modelo de uma rede do tipo Senquantial com 7 camadas
model = Sequential()

model.add(Dense(units= 1024, activation='relu', input_shape=X_train.shape[1:]))
model.add(Dropout(rate=0.3))
model.add(Dense(units= 512, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units= 512, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units= 1, activation= 'sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, Y_train, epochs=10, validation_data=(x_test, y_test), verbose=1)

Os valores obtidos acima foram também bons, com a acurácia acima de 86%. Entretanto, analisando sua evolução, é possível perceber que houve um pequeno overfit sobre o conjunto de treino. Sua evolução pode ser vista nos gráficos abaixo. Neles, verificamos que houve um aumento do *loss* de validação com as épocas.

In [0]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['acc'],
                    mode='lines',
                    name='training acc'),
                    row=1,
                    col=1)
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['val_acc'],
                    mode='lines',
                    name='validation acc'),
                    row=1,
                    col=1)
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['loss'],
                    mode='lines',
                    name='training loss'),
                    row=1,
                    col=2)
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['val_loss'],
                    mode='lines',
                    name='validation loss'),
                    row=1,
                    col=2)